In [6]:
"""Example vacuum minimization with pyCHARMM"""

import os
import shutil
import subprocess
from functools import partial

In [7]:
# Create output directory
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

In [8]:
# Helper functions

# Run a command in the shell
run = partial(subprocess.run, check=False, text=True, capture_output=True)

In [9]:
# Path to alphafold model of CazE
af_model_path = "../../alphafold_runs/models/caze.pdb"

In [11]:
# Step 1: Align structure to 2e1t_A.pdb
aln_target_path = "../2e1t_A.pdb"
aln_dir = os.path.join(output_dir, "align")
os.makedirs(aln_dir, exist_ok=True)

# Path to TMalign
TMALIGN_EXEC = "../TMalign"


# Align structures to target
def align_to_target(
    mobile,
    target,
    out_prefix,
):

    aln_cmd = [
        TMALIGN_EXEC,
        mobile,
        target,
        "-o",
        out_prefix,
    ]

    run(aln_cmd)

    # Check if file generate
    outfile = out_prefix + ".pdb"
    assert os.path.isfile(outfile), f"File not found: {outfile}"


# Align alphafold model to target
align_to_target(
    af_model_path,
    aln_target_path,
    os.path.join(aln_dir, "caze"),
)

In [13]:
# Run MMTSB convpdb to convert to CHARMM format
# Convpdb by default sets all histidines to HSD, which is ok for our case
pdb_conv_path = os.path.join(output_dir, "caze_conv.pdb")

# Check if convpdb is in path
CONVPDB_EXEC = "convpdb.pl"
assert shutil.which(CONVPDB_EXEC), f"convpdb.pl not found in path"


def run_convpdb(infile, outfile):

    convpdb_cmd = [
        CONVPDB_EXEC,
        "-out",
        "charmm22",
        "-nohetero",
        "-segnames",
        "-nsel",
        "A:",
        infile,
    ]

    out = run(convpdb_cmd)

    # Check return code
    assert out.returncode == 0, f"convpdb failed: {out.stderr}"

    # Write output to file
    with open(outfile, "w") as f:
        f.write(out.stdout)


# Convert TMalign output to CHARMM format
run_convpdb(
    os.path.join(aln_dir, "caze.pdb"),
    pdb_conv_path,
)

In [16]:
# Minimize converted pdb with pyCHARMM

# Path to CHARMM_LIB_DIR
CHARMM_LIB_DIR = "/home/azamh/charmm/08302023/build_pyCHARMM/install/lib"

# Path to run script
MIN_SCRIPT = "../vacuum_min.py"

# Conda environment
CONDA_ENV = "anc_acyl"

# Toppar directory
TOPPAR_DIR = "../../toppar"


def minimize_pdb(
    input_pdb,
    output_pdb,
    output_psf,
):

    # Run minimization
    cmd = [
        "conda",
        "run",
        "-n",
        CONDA_ENV,
        "python",
        MIN_SCRIPT,
        "-i",
        input_pdb,
        "-o",
        output_pdb,
        "-p",
        output_psf,
        "--charmm_lib_dir",
        CHARMM_LIB_DIR,
        "--toppar_dir",
        TOPPAR_DIR,
    ]

    cp = run(cmd)

    print(cp.stdout)
    print(cp.stderr)

    # Check if files generated
    assert os.path.isfile(output_pdb), f"File not found: {output_pdb}"
    assert os.path.isfile(output_psf), f"File not found: {output_psf}"


# Minimize
minimize_pdb(
    pdb_conv_path,
    os.path.join(output_dir, "caze_min.pdb"),
    os.path.join(output_dir, "caze_min.psf"),
)

Arguments:
input output/caze_conv.pdb
output output/caze_min.pdb
psf output/caze_min.psf
charmm_lib_dir /home/azamh/charmm/08302023/build_pyCHARMM/install/lib
toppar_dir ../../toppar
  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::../../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name ../../toppar/par_all36m_prot.prm -
 CHARMM>     flex
 VOPEN> Attempting to open::../../toppar/par_all36m_prot.prm::

          PARAMETER FILE BEING READ